In [25]:
import requests
import time


def importrangeAllowAccessAutomation(get_spreadsheet_id, give_spreadsheet_id, creds):
    url = f'https://docs.google.com/spreadsheets/d/{get_spreadsheet_id}/externaldata/addimportrangepermissions?donorDocId={give_spreadsheet_id}&includes_info_params=true&cros_files=false'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {creds.token}',
    }
    # 데이터를 JSON 형태로 변환하고, POST 요청을 보냅니다.
    response = requests.post(url, headers=headers)
    # 응답 확인
    if 'application/json' in response.headers.get('Content-Type', ''):
        try:
            data = response.json()
            print(data)
        except ValueError:
            print("JSON 디코딩 실패", response.text)
    else:
        print("응답이 JSON 형식이 아닙니다:", response.text)

    

def importrange_invoice_to_seller_ordersheet(invoice_sheet_id, TitleAndSpreadsheets_ids, creds):
    for spreadsheet_id in TitleAndSpreadsheets_ids:
        print(spreadsheet_id[0], "importrange_invoice_to_seller_ordersheet")
        importrangeAllowAccessAutomation(spreadsheet_id[1], invoice_sheet_id, creds)



In [6]:
# 특정 셀에 수식을 적용하는 함수
def apply_formula_to_cell_sheet2(service, spreadsheet_id, invoice_sheet_id):
    cell_range = 'A1'  # 수식을 적용할 셀 범위
    formula = f'=IMPORTRANGE("{invoice_sheet_id}","\'시트1\'!A1:M999")'  # A2 셀에 적용할 수식
    range_name = f"상품목록!{cell_range}"  # 지정된 시트의 특정 셀 범위
    value_range_body = {
        'values': [[formula]]  # 적용할 수식
    }
    
    request = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=range_name,
        valueInputOption='USER_ENTERED',
        body=value_range_body
    )
    request.execute()

In [2]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
            #   'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [1]:
def get_values_from_range(service, spreadsheet_id, range_name):
    # Google Sheets API를 사용하여 특정 범위의 값 가져오기
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    # 값이 있는지 확인 후 반환
    values = result.get('values', [])

    return values
    

In [41]:
user_sheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
# spreadsheet_copy_id = "1p0WM4X2JztS1LfGKVdUXd3GTvBJJGYPNO0ya8ihfns4"
import time
import random

if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 
    # range_name = "셀러컨택포인트!A86:T87"
    # range_name = "셀러컨택포인트!A93:T94"
    # range_name = "셀러컨택포인트!A186:T192"

    # range_name = "셀러컨택포인트!A292:T292"

    # range_name = "셀러컨택포인트!A193:T284"

    # range_name = "셀러컨택포인트!A193:T292"

    # range_name = "셀러컨택포인트!A327:T392"

    # range_name = "셀러컨택포인트!A293:T392"

    # range_name = "셀러컨택포인트!A393:T492"

    range_name = "셀러컨택포인트!A493:T503"
    
    user_data_list = get_values_from_range(service_sheets, user_sheet_id, range_name)

    user_ids = [[item[1], item[10]] for item in user_data_list]
    print(user_ids)

    # product_sheet_id = "16t8okayXoD3jdyiBFJ4w43fNA3o5n7rONfH8aVF7iWg" #111210
    #211~310:
    # product_sheet_id = "1XDBIK9o2k4AVMcW2gfieDZ4qSBxcTW8npw596AY6KCY"
    # # #311~410:
    # product_sheet_id = "1kiYHm2CLEvKxiE9Fv2j1F7vA4DDem4DROuU26d2RXBo"
    # # #411~510:
    # product_sheet_id = "1kiYHm2CLEvKxiE9Fv2j1F7vA4DDem4DROuU26d2RXBo"
    # # #511~610:
    product_sheet_id = "1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4"

    for spreadsheet_id in user_ids:
        if spreadsheet_id == '':
            continue
        time.sleep(random.uniform(0.5, 1.5))
        print(spreadsheet_id[0], "apply_formula_to_cell_sheet2")
        apply_formula_to_cell_sheet2(service_sheets, spreadsheet_id[1], product_sheet_id)

    importrange_invoice_to_seller_ordersheet(product_sheet_id, user_ids, creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=520657704422-o724cfbjdm5vnt6iko73r4buo8gjr68k.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fscript.projects+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file&state=dAXog9GvQrfoZoIGiaKd3OU51nlIQX&access_type=offline
[['511', '12l-0fx_ygkUwkPbzZSb6KvEobUabyU1HodDBftdfdTc'], ['512', '1PHpg2H2W-ChCOsJyIOebqf7LfcryrOBgHG5D6AYFIHI'], ['513', '1_eWdxS4Xdykn-GbZTTyOdH8f41GEvwURRcisG-PI9XU'], ['514', '1Z0BJDjKsvPOrYXKD93NyRLUu--ecxpNue05n_UPLJjc'], ['515', '1-0alpGTk3TATHYuDJn3-vtlTpJ40hRrQYY0aINWUjLk'], ['516', '1VhHatV-asswfTDa9M13B8Orf2aDSmbdYWBzwrI4dqjM'], ['517', '133IbYEp-aXQH-ECojwEEak6LKRgPKraS0yqTSvqNvWM'], ['518', '1Ycvhq1jXkg9XDTtGoXiFoPRggMLHjCXQlFxlxD__LPM'], ['519', '1DQ9qzOMVa5ic1